In [1]:
import csv
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

raw_train_df = pd.read_csv('fake_or_real_news_training.csv')
raw_test_df = pd.read_csv('fake_or_real_news_test.csv')

raw_train_df.head()


,ID,title,text,label,X1,X2
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,NaN,NaN
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,NaN,NaN
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,NaN,NaN
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,NaN,NaN
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,NaN,NaN


In [2]:
raw_train_df.shape

(3999, 6)

In [3]:
raw_test_df.shape

(2321, 3)

In [4]:
raw_test_df.head()

,ID,title,text
0,10498,September New Homes Sales Rise——-Back To 1992 ...,September New Homes Sales Rise Back To 1992 Le...
1,2439,Why The Obamacare Doomsday Cult Can't Admit It...,But when Congress debated and passed the Patie...
2,864,"Sanders, Cruz resist pressure after NY losses,...",The Bernie Sanders and Ted Cruz campaigns vowe...
3,4128,Surviving escaped prisoner likely fatigued and...,Police searching for the second of two escaped...
4,662,Clinton and Sanders neck and neck in Californi...,No matter who wins California's 475 delegates ...


In [5]:
prepared_train_df = raw_train_df.copy()
prepared_test_df = raw_test_df.copy()

In [6]:
prepared_test_df['label'] = None
prepared_test_df.head()

,ID,title,text,label
0,10498,September New Homes Sales Rise——-Back To 1992 ...,September New Homes Sales Rise Back To 1992 Le...,None
1,2439,Why The Obamacare Doomsday Cult Can't Admit It...,But when Congress debated and passed the Patie...,None
2,864,"Sanders, Cruz resist pressure after NY losses,...",The Bernie Sanders and Ted Cruz campaigns vowe...,None
3,4128,Surviving escaped prisoner likely fatigued and...,Police searching for the second of two escaped...,None
4,662,Clinton and Sanders neck and neck in Californi...,No matter who wins California's 475 delegates ...,None


# Train/Test Split of Data

In [9]:
def score_model(df, model, vectorizer, target_var, independent_var, 
                size, seed, return_model=True):
    df_new = df.copy()

    X = df_new.loc[:, independent_var]
    y = df_new.loc[:, target_var]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=size, random_state=seed)

    pipeline = make_pipeline(vectorizer, model).fit(X=X_train, y=y_train)
    pred = pipeline.predict(X_test)

    if return_model == False:
        return(pipeline.score(X_test, y_test), y_test, pred)
    else:
        return(pipeline.score(X_test, y_test), y_test, pred, pipeline)


def cv_evaluate(df, model, vectorizer, target_var, independent_var, seed, cv, 
                standalone=False, metric='accuracy'):
    df_new = df.copy()
    kfolds = KFold(n_splits=cv, shuffle=True, random_state=seed)

    X = df_new.loc[:, independent_var]
    y = df_new.loc[:, target_var]

    if standalone == True:
        benchmark_model = make_pipeline(
            vectorizer, model).fit(X=X, y=y.values.ravel())
    else:
        benchmark_model = make_pipeline(model).fit(X=X, y=y.values.ravel())

    scores = cross_val_score(benchmark_model, X, y.values.ravel(),
                             scoring=metric, cv=kfolds)
    return scores[scores >= 0.0]


def confusion_heat_map(test_set, prediction_set, class_names):
    cm = confusion_matrix(test_set, prediction_set, class_names)
    fig, ax = plt.subplots()

    # create heatmap
    g = sns.heatmap(cm, annot=True, cmap="YlGnBu", fmt='g',
                    xticklabels=True, yticklabels=True)
    ax.xaxis.set_label_position("top")
    ax.set_xticklabels(class_names)
    ax.set_yticklabels(class_names)
    plt.title('Confusion matrix', y=1.1)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


def run_full_iteration(df, model, vectorizer, target_var, independent_var, size, seed, cv=5):
    score, y_test, pred, model = score_model(
        df, model, vectorizer, target_var, independent_var, size, seed)
    print("Accuracy: %0.3f" % score)

    accuracy = cv_evaluate(df, model, vectorizer,
                           target_var, independent_var, seed, cv)
    print('Mean Accuracy after CV: {:.3f} +/- {:.03f}'.format(np.mean(accuracy), np.std(accuracy)))
    print('Best Accuracy after CV: {:.3f}'.format(max(accuracy)))

    confusion_heat_map(y_test, pred, ['FAKE', 'REAL'])


In [10]:
# Set test set size, seed and target metric for the rest of the pipeline
size = 0.20
seed = 12345
target_var = 'label'
independent_var = 'text'

models = []
models.append(('MNB', MultinomialNB()))
models.append(('LR', LogisticRegression(solver='lbfgs', multi_class='auto')))
# models.append(('KNN', KNeighborsClassifier())) #very poor performance
models.append(('RFC', RandomForestClassifier(random_state=seed)))
models.append(('SVM', SVC(kernel='linear')))
# models.append(('GB', GradientBoostingClassifier())) #takes too long

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)

# evaluate each model in turn
results = []
names = []
for name, model in models:
    cv_results = cv_evaluate(prepared_train_df, model,
                             vectorizer, target_var, 
                             independent_var, seed, cv=4, standalone=True)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)


ValueError: Found input variables with inconsistent numbers of samples: [5, 3999]

In [ ]:
vectorizer = CountVectorizer(stop_words='english')

models = []
models.append(('MNB', MultinomialNB()))
# models.append(('LR', LogisticRegression(solver='lbfgs', multi_class='auto'))) #Fails to converge
# models.append(('KNN', KNeighborsClassifier())) #very poor performance
models.append(('RFC', RandomForestClassifier(random_state = seed)))
models.append(('SVM', SVC(kernel='linear')))
# models.append(('GB', GradientBoostingClassifier())) #takes too long

# evaluate each model in turn
results = []
names = []
for name, model in models:
    cv_results = cv_evaluate(prepared_train_df, model,
                             vectorizer, target_var, seed, cv=4, standalone=True)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)


# Full Analysis of Best Model

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)
model = SVC(kernel='linear')

run_full_iteration(prepared_train_df, model, vectorizer, target_var, size, seed, cv=4)


# Feature Engineering

## FE1: Combine Title and Text

In [ ]:
fe1_train_df = prepared_train_df.copy()
fe1_test_df = fe1_test_df.copy()

fe1_train_df['combined'] = fe1_train_df['title'] + fe1_train_df['text']
fe1_test_df['combined'] = fe1_test_df['title'] + fe1_test_df['text']
fe1_train_df.head()

In [ ]:
independent_var = 'combined'
model = LogisticRegression(solver='lbfgs', multi_class='auto')
run_full_iteration(fe1_train_df, model, vectorizer,
                   target_var, independent_var, size, seed)
